# Time Series Forecasting (Synthetic)

This notebook forecasts daily sales using synthetic data. No real data is used.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.exponential_smoothing import ExponentialSmoothing


In [ ]:
df = pd.read_csv('synthetic_daily_sales.csv', parse_dates=['date'])
df = df.sort_values('date')
df['dow'] = df['date'].dt.dayofweek
for k in [3,7,14]:
    df[f'y_lag{k}'] = df['sales'].shift(k)
df = df.dropna().reset_index(drop=True)
df.head()

## Baseline: ETS/Exponential Smoothing

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing as HWES
series = df.set_index('date')['sales']
train = series.iloc[:-60]
test = series.iloc[-60:]
model = HWES(train, trend='add', seasonal='add', seasonal_periods=7).fit()
fcst = model.forecast(len(test))
mae = mean_absolute_error(test, fcst)
mape = mean_absolute_percentage_error(test, fcst)
print({'ETS_MAE': round(mae,2), 'ETS_MAPE': round(mape,4)})
plt.figure()
plt.plot(series.index, series.values, label='Actual')
plt.plot(test.index, fcst.values, label='ETS Forecast')
plt.legend()
plt.title('Daily Sales & ETS Forecast')
plt.show()

## Feature Model: Random Forest Regressor

In [ ]:
feature_cols = ['min_temp','precip','is_holiday','dow','y_lag3','y_lag7','y_lag14']
X = df[feature_cols]
y = df['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
rf = RandomForestRegressor(n_estimators=300, random_state=42)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
mae = mean_absolute_error(y_test, pred)
mape = mean_absolute_percentage_error(y_test, pred)
print({'RF_MAE': round(mae,2), 'RF_MAPE': round(mape,4)})
imp = pd.Series(rf.feature_importances_, index=feature_cols).sort_values(ascending=False)
print(imp)
plt.figure()
plt.plot(df.loc[X_test.index, 'date'], y_test.values, label='Actual')
plt.plot(df.loc[X_test.index, 'date'], pred, label='RF Pred')
plt.legend()
plt.title('Random Forest Forecast (Holdout)')
plt.show()